In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

In [2]:
data= pd.read_csv('Dataset.csv')

In [3]:
data.sample(5)

,Unnamed: 0,age,weight(kg),height(m),gender,BMI,BMR,activity_level,calories_to_maintain_weight,BMI_tags,Label
6438,13399,28,67.629682,1.631374,M,25.411499,1617.814,1.7,2750.2838,9,31
9953,22418,88,98.749607,1.630980,M,37.122583,1635.960,1.5,2453.9400,9,30
4981,9766,19,57.616348,1.632490,F,21.619458,1396.340,1.3,1815.2420,8,16
7636,15846,33,75.268830,1.631805,M,28.266923,1688.686,1.5,2533.0290,9,30
4634,9207,21,57.000203,1.632301,M,21.393197,1520.253,1.7,2584.4301,8,20


In [4]:
data = data.drop(["Unnamed: 0", "BMI_tags", "Label"], axis=1)

In [5]:
data = pd.get_dummies(data, columns=['gender'], drop_first=False)

In [6]:
data.sample(5)

,age,weight(kg),height(m),BMI,BMR,activity_level,calories_to_maintain_weight,gender_F,gender_M
3574,16,52.376701,1.594365,20.604530,1353.658,1.9,2571.9502,True,False
1067,8,23.040823,1.175735,16.667839,915.127,1.9,1738.7413,False,True
9593,43,92.803060,1.632253,34.832724,1861.128,1.2,2233.3536,False,True
6291,27,66.080631,1.630706,24.849787,1603.058,1.5,2404.5870,False,True
3411,15,54.039520,1.597050,21.187248,1502.866,1.7,2554.8722,False,True


In [7]:
data['activity_level'] = (data['activity_level'] - 1) * 100

In [8]:
data['gender_F'] = data['gender_F'] * 1
data['gender_M'] = data['gender_M'] * 1

In [9]:
numeric_data = data.select_dtypes(include='number')
data = data.drop('activity_level', axis=1)

In [10]:
numeric_data.columns

Index(['age', 'weight(kg)', 'height(m)', 'BMI', 'BMR', 'activity_level',
       'calories_to_maintain_weight', 'gender_F', 'gender_M'],
      dtype='object')

In [24]:
numeric_data.sample(3)

,age,weight(kg),height(m),BMI,BMR,activity_level,calories_to_maintain_weight,gender_F,gender_M
9332,44,93.988691,1.631272,35.320195,1870.081,20.0,2244.0972,0,1
5822,27,66.679507,1.630818,25.071575,1445.462,50.0,2168.1930,1,0
9861,55,96.405433,1.632867,36.157623,1829.188,20.0,2195.0256,0,1


In [23]:
numeric_data.to_csv('numeric_data.csv', index=False)

In [11]:
X = numeric_data.drop(['calories_to_maintain_weight'], axis=1)  
y = numeric_data['calories_to_maintain_weight']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
print(f'100% of our data: {len(data)}.')
print(f'80% for training data: {len(X_train)}.')
print(f'20% for test data: {len(X_test)}.')

100% of our data: 10726.
80% for training data: 8580.
20% for test data: 2146.


In [14]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [15]:
predictions = model.predict(X_test)

In [16]:
r2_3 = r2_score(y_test, predictions)
RMSE_3 = mean_squared_error(y_test, predictions, squared=False)
MSE_3 = mean_squared_error(y_test, predictions)
MAE_3 = mean_absolute_error(y_test, predictions)

In [17]:
print("R2 = ", round(r2_3, 4))
print("RMSE = ", round(RMSE_3, 4))
print("The value of the metric MSE is ", round(MSE_3, 4))
print("MAE = ", round(MAE_3, 4))

R2 =  0.987
RMSE =  45.3786
The value of the metric MSE is  2059.2173
MAE =  32.4556


In [18]:
def calculate_bmi(weight_kg, height_m):
    return weight_kg / (height_m ** 2)

In [19]:
data.sample(5)

,age,weight(kg),height(m),BMI,BMR,calories_to_maintain_weight,gender_F,gender_M
6936,31,73.853549,1.632041,27.727402,1683.014,2187.9182,0,1
9782,83,97.253413,1.632700,36.483147,1650.322,2145.4186,0,1
7356,31,75.211026,1.631223,28.265384,1701.203,2211.5639,0,1
10457,62,96.867880,1.631743,36.381157,1570.928,2042.2064,1,0
5300,24,60.463319,1.631533,22.714335,1546.914,2629.7538,0,1


In [20]:
def calculate_bmr(age, weight_kg, height_m, gender_F, gender_M):
    if gender_F == 1:  # Female
        return 655 + (9.6 * weight_kg) + (1.8 * height_m * 100) - (4.7 * age)
    elif gender_M == 1:  # Male
        return 66 + (13.7 * weight_kg) + (5 * height_m * 100) - (6.8 * age)
    else:
        raise ValueError("Invalid gender values")

In [21]:
def calculate_daily_calories_direct(user_inputs, initial_weight, desired_weight, time_interval, modelLR):
    # Extract user inputs
    age = user_inputs['age']
    weight_kg = user_inputs['weight']
    height_m = user_inputs['height']
    gender_F = user_inputs['gender_F']
    gender_M = user_inputs['gender_M']
    activity_level = user_inputs['activity_level']

    # Calculate BMI and BMR using the provided formulas
    BMI = calculate_bmi(weight_kg, height_m)
    BMR = calculate_bmr(age, weight_kg, height_m, gender_F, gender_M)

    # Calculate total weight change
    weight_change = desired_weight - initial_weight
    calories_per_kg = 7700

    # Estimated calories per kg change (adjust as needed)
    daily_weight_change = weight_change / time_interval

    # Calculate caloric difference based on weight change goal
    caloric_difference = daily_weight_change * calories_per_kg

    # Create input array for model prediction
    input_array = np.array([[age, weight_kg, height_m, BMI, BMR, activity_level, gender_F, gender_M]])

    # Use the trained model to predict daily maintenance calories
    model_predicted_calories = modelLR.predict(input_array)

    # Sum the model predicted calories and caloric difference
    daily_cal = model_predicted_calories + caloric_difference

    return daily_cal


In [22]:
user_inputs = {}
user_inputs['age'] = float(input("Enter your age: "))
user_inputs['weight'] = float(input("Enter your weight in kg: "))
user_inputs['height'] = float(input("Enter your height in meters: "))
user_inputs['gender_F'] = int(input("Are you a woman? (1 for Yes, 0 for No): "))

# Calculate the opposite value for gender_M
user_inputs['gender_M'] = 1 if user_inputs['gender_F'] == 0 else 0

user_inputs['activity_level'] = float(input("""
Enter your activity level between 0 and 1:
0   - 0 minutes of exercise per day
10  - 1 hour of exercise per day
20  - 2 hours of exercise per day
30  - 3 hours of exercise per day
40  - 4 hours of exercise per day
50  - 5 hours of exercise per day
60  - 6 hours of exercise per day
70  - 7 hours of exercise per day
80  - 8 hours of exercise per day
90  - 9 hours of exercise per day
100 - 10 hours of exercise per day
"""))

initial_weight = user_inputs['weight']
desired_weight = float(input("Enter your desired weight in kg: "))
time_interval = float(input("Enter the time interval in days to reach your desired weight: "))

# Call the function to calculate daily calories
result = calculate_daily_calories_direct(user_inputs, initial_weight, desired_weight, time_interval, model)
print(f"\nEstimated daily calories: {result[0]}")

Enter your age:  28
Enter your weight in kg:  76
Enter your height in meters:  1.74
Are you a woman? (1 for Yes, 0 for No):  0

Enter your activity level between 0 and 1:
0   - 0 minutes of exercise per day
10  - 1 hour of exercise per day
20  - 2 hours of exercise per day
30  - 3 hours of exercise per day
40  - 4 hours of exercise per day
50  - 5 hours of exercise per day
60  - 6 hours of exercise per day
70  - 7 hours of exercise per day
80  - 8 hours of exercise per day
90  - 9 hours of exercise per day
100 - 10 hours of exercise per day
 80
Enter your desired weight in kg:  80
Enter the time interval in days to reach your desired weight:  90



Estimated daily calories: 3381.5125986847493


C:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
